<center>Ángel Ríos San Nicolás</center>

# Claves secretas
## Parámetros para EC-DSA
Construimos el grupo $\left(E,\oplus\right)$ de la curva elíptica $E$ dada por $y=x^3+7$ sobre el cuerpo finito $\mathbb{F}_p$ con $p=2^{256} - 2^{32} - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1$ primo.

$\rightarrow$¿Cuántos puntos tiene esta curva? Calcúlalo con Sage y expresa esa cantidad en base 16. Comprueba que
se verifica el teorema de Hasse.

In [1]:
p = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1
E = EllipticCurve(GF(p), [0, 7])  # Curva elíptica y^2 = x ^ 3 + 7
N = E.cardinality()  # Número de puntos de la curva
print("Número de puntos de E en hexadecimal: ", hex(N))

Número de puntos de E en hexadecimal:  0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141


Verificamos que se cumple el teorema de Hasse, que el orden $N$ de la curva no dista de $p+1$ más de $2\sqrt{p}$, es decir, que se cumple
$$p+1-\sqrt{p}< N < p+1+2\sqrt{p}.$$

In [2]:
# Teorema de Hasse. El número de puntos de la curva está en (p+1-2sqrt(p), p+1+2sqrt(p))
if p+1-2*sqrt(p) < N:
    print(True)
if p+1+2*sqrt(p) > N:
    print(True)

True
True


Consideramos el punto dado por
$$\begin{array}{rl}G = & (79BE667E\;F9DCBBAC\;55A06295\; CE870B07\; 029BFCDB\; 2DCE28D9\; 59F2815B\; 16F81798,\\
& 483ADA77\; 26A3C465\; 5DA4FBFC\; 0E1108A8\; FD17B448\; A6855419\; 9C47D08F\; FB10D4B8).
\end{array}
$$

$\rightarrow$ Comprueba que se trata, efectivamente, de un punto de la curva. ¿Cuál es el orden del subgrupo que genera?
Comprueba que ese orden es un número primo, como requiere el protocolo DSA.

Guardamos las coordenadas del punto en decimal.

In [3]:
Gx = int("79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798", base = 16) % p
Gy = int("483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8", base = 16) % p

Construimos $G$ como punto de la curva y comprobamos que está bien definido.

In [4]:
G = E((Gx, Gy))
G.parent()

Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 7 over Finite Field of size 115792089237316195423570985008687907853269984665640564039457584007908834671663

Calculamos el orden del subgrupo $<G>$ de $E$ generado por $G$ y comprobamos que es primo.

In [5]:
ordenG = G.order()
print("Orden del subgrupo generado por G: ", ordenG)
ordenG.is_prime()

Orden del subgrupo generado por G:  115792089237316195423570985008687907852837564279074904382605163141518161494337


True

$\rightarrow$ El ejecutable _01_claves_ej_1_ muestra una pareja de claves. Utiliza Sage para comprobar que casan.

Guardamos la clave privada $d$ y las coordenadas de la clave pública $(x,y)$. Tenemos que ver que casan, es decir, que la clave pública es $(x,y)=d\cdot G$. Lo comprobamos coordenada a coordenada.

In [6]:
d = int("0x45737461206672617365206D65206C612067756172646F2070617261206DED2E", base = 16) % p
x = int("0x1693AAC9007A1C28B2DF4BEB207ECD6C6A150C324E2066897252756F3BFC375A", base = 16) % p
y = int("0x7F3EF84CC4BBD9C6988E3F77FE38127CF5B8C1513FC399CA26406774722DE01A", base = 16) % p
Q = d * G
print(Q[0] == x)
print(Q[1] == y)

True
True


Como se cumplen las igualdades, las claves casan y $(x,y)$ es la clave pública asociada a la clave privada $d$.

$\rightarrow$ Si $p$ es impar y $(x,y)\in\mathbb{F}_p^2$ verifica $y^2=x^3+ax+b$, ¿se puede calcular $y$ a partir de $x$?

Calcular $y$ a partir de $x$ consiste en calcular las raíces cuadradas de $x^3+ax+b$ en $\mathbb{F}_p$. Como $y^2$ es un cuadrado por hipótesis siempre se puede calcular una raíz cuadrada $\alpha\in\mathbb{F}_p$, por ejemplo con el algoritmo de Tonelli-Shanks. Sin embargo, en general no podemos calcular $y$ porque si $\alpha\in\mathbb{F}_p$ es una raíz cuadrada de $y$, entonces $\alpha^2=y\mod p$, pero también $(-\alpha)^2=y\mod p$ por lo que tenemos dos raíces cuadradas $\alpha,-\alpha\in\mathbb{F}_p$ y no podemos distinguir cuál es $y$.  Vamos a ver que siempre una raíz es par y la otra es impar. Como $p$ es impar, $p=1\mod p$. Sin pérdida de generalidad suponemos el signo solo en $\alpha$. Si $\alpha$ es par, $\alpha=0\mod 2$, entonces $$-\alpha=p-\alpha=1-\alpha=1-0=1\mod 2$$ con lo que $-\alpha$ es impar. Por el contrario si $\alpha$ es impar, $\alpha=1\mod 2$, entonces $$-\alpha=p-\alpha=1-1=0\mod 2$$ con lo que $-\alpha$ es par. Esto permite hallar $y$ a partir de $x$ y la paridad de $y$.

Como lo vamos a necesitar más adelante, describimos un algoritmo sencillo para calcular raíces cuadradas módulo $p$ si existen y $p=3\mod 4$.
Dado $a\in\mathbb{F}_p$, buscamos $n\in\mathbb{N}$ tal que $a^n=x$ con $x^2=a$. En particular $a^{4n}=a^2$, es decir, $a^{4n-2}=1\mod p$. Por el teorema de Euler-Fermat, también $a^{p-1}=1\mod p$. Por lo tanto, debe ser $4n-2=p-1$, es decir, $n=\frac{p+1}{4}$ que es natural si y solo si $p+1=0\mod 4$, esto es, $p=3\mod 4$. Por lo tanto, si $a\in\mathbb{F}_p$ es un cuadrado y $p=3\mod 4$, entonces $a^{\frac{p+1}{4}}$ y su opuesto son las raíces cuadradas de $a$ módulo $p$.

Comprobamos que el primo $p$ sobre el que se define la curva _secp256k1_ es $3$ módulo $4$ con lo que podemos calcular las raíces cuadradas fácilmente mediante una potencia.

In [11]:
p % 4

3

$\rightarrow$ Si $(x,y)$ está en la curva _secp256k1_ y conocemos $y$, ¿podemos determinar $x$?

Si $(x,y)$ está en la curva _secp256k1_, cumple $y^2=x^3+7$. Determinar $x$ a partir de $y$ consiste en hallar las raíces cúbicas de $y^2-7$ en $\mathbb{F}_p$. Podemos calcular las raíces cúbicas de la unidad módulo $p$ resolviendo la ecuación $x^3-1=0$ en $\mathbb{F}_p$, es decir,
$$x^3-1=\left(x-1\right)\left(x^2+x+1\right)$$
Por lo tanto, $x=1$ o $x=\left(-1\pm\sqrt{1-4}\right)2^{-1}=\left(-1\pm\sqrt{-3}\right)2^{-1}$ donde $\sqrt{-3}$ representa una de las raíces cuadradas de $-3$ que se puede calcular con el algoritmo anterior como $(-3)^{\frac{p+2}{4}}$ y $2^{-1}$ es el inverso de $2$ que se puede calcular con el algoritmo de Euclides extendido. De esta manera tenemos $\alpha=\left(-1\pm\sqrt{-3}\right)2^{-1}$ una raíz primitiva cúbica de la unidad.

Sean $1,\alpha,\alpha^2$ las raíces cúbicas de la unidad. Sabemos que las raíces cúbicas de $y^2-7$ son $\sqrt[3]{y^2-7}$, $\sqrt[3]{y^2-7}\alpha$, $\sqrt[3]{y^2-7}\alpha^2$ donde $\sqrt[3]{y^2-7}$ denota una raíz tercera de la unidad. Por lo tanto, simplemente con $y$ no podemos distinguir cuál de las raíces cúbicas es $x$, no se puede calcular.

Describimos un algoritmo sencillo para calcular una raíz cúbica módulo $p$ si existe y $p=7\mod 9$. Dado $a\in\mathbb{F}_p$, buscamos $n\in\mathbb{N}$ tal que $a^n=x$ con $x^3=a$. En particular $a^{9n}=a^3$, es decir, $a^{9n-3}=1\mod p$. Por el teorema de Euler-Fermat, también $a^{p-1}=1\mod p$. Por lo tanto, debe ser $9n-3=p-1$, es decir, $n=\frac{p+2}{9}$ que es natural si y solo si $p+2=0\mod 9$, esto es, $p=7\mod 9$. Por lo tanto, si $a\in\mathbb{F}_p$ es un cubo y $p=3\mod 9$, entonces $a^{\frac{p+2}{9}}$ es una raíz cúbica módulo $p$.

El primo $p$ sobre el que se define la curva _secp256k1_ es $7$ módulo $9$.

In [31]:
print(p % 9)

7


$\rightarrow$ He generado un par de claves de manera aleatoria, utilizando _01_claves_. La clave pública ha quedado
guardada en el fichero _publica_02.raw_ (que se muestra a continuación y está disponible junto con el
código que acompaña esta hoja). A partir de los datos de este fichero en concreto (que no de otro que generes),
calcula las dos componentes del punto $Q = d · G$ correspondiente.

El fichero 01_publica contiene los bytes de una clave pública, es decir, un punto de la curva.

Los primeros dos bytes son la longitud del bloque en bytes y el tipo de codificiación del punto. El resto de bytes guardan las coordenadas. Como la $x$ y la $y$ se representan igual, tienen el mismo número de dígitos.

La salida es:

00000000: 4104 1693 aac9 007a 1c28 b2df 4beb 207e  A......z.(..K. ~

00000010: cd6c 6a15 0c32 4e20 6689 7252 756f 3bfc  .lj..2N f.rRuo;.

00000020: 375a 7f3e f84c c4bb d9c6 988e 3f77 fe38  7Z.>.L......?w.8

00000030: 127c f5b8 c151 3fc3 99ca 2640 6774 722d  .|...Q?...&@gtr-

00000040: e01a  

Los dos primeros bytes 41 indican en hexadecimal el número de bytes del bloque. En este caso $4\cdot 16 + 1 =65$, es decir $2\cdot 65=130$ símbolos. 04 indica el tipo de codificación del punto, en este caso que el punto se guarda con sus coordenadas x e y. El resto de bytes guardan primero el valor de x y después el valor de y, como x e y están codificados igual, la primera mitad, $64$ símbolos, se corresponde a x y la segunda, los últimos $64$ símbolos, a y.

$$x = 1693\; aac9\; 007a\; 1c28\; b2df\; 4beb\; 207e\; cd6c\; 6a15\; 0c32\; 4e20\; 6689\; 7252\; 756f\; 3bfc\; 375a$$
$$y = 7f3e\; f84c\;c4bb\;d9c6\;988e\;3f77\;fe38\;127c\;f5b8\;c151\;3fc3\;99ca\;2640\;6774\;722d\;e01a$$

Si tenemos

2103EA1901C0F7B9FB4F22F187CED808141EE30714ADEC3841A7C5209735E0E081E0,

sabemos que tenemos el valor de $x$ y que $y$ es impar. Para enontrar el valor de $y$, debemos hacer lo de antes, calcular las raíces cuadradas de $x^3+7$. Sabemos que tienen distinta paridad y la que buscamos es la raíz impar.

La podemos calcular aplicando la función **sqrt** de Sage.

In [101]:
xq = int("EA1901C0F7B9FB4F22F187CED808141EE30714ADEC3841A7C5209735E0E081E0", base=16) % p  # Valor de x.
yq = sqrt(GF(p)(xq^3+7))
yq = ZZ(yq)
print("y= ", yq)  # Como y es impar, es el que estábamos buscando.
Q = E((xq,yq))  # Es un punto de la curva.
print(Q.parent())

y=  7976399251461369506354258185750123002182989095567710819611729233031888812835
Abelian group of points on Elliptic Curve defined by y^2 = x^3 + 7 over Finite Field of size 115792089237316195423570985008687907853269984665640564039457584007908834671663


Mostramos que también la podíamos haber calculado simplemente elevando a $\frac{p+1}{4}$ como mostramos previamente.

In [96]:
yq2 = pow(xq^3+7,(p+1)/4,p)
yq == yq2

True

$\rightarrow$ Determina ahora el valor de $d$ que se corresponde con esos datos.

Tenemos los datos de $G$ y la clave pública $(x,y)$ tal que $(x,y)=d\cdot G$. Recuperar $d$ consiste en resolver el problema del logaritmo discreto de $(x,y)$ en base $G$ en el grupo de la curva elíptica $\left(E,\oplus\right)$. Sabemos que este problema, como el de la mochila, es difícil de resolver computacionalmente y que no somos capaces de calcularlo eficientemente en la práctica. La dificultad del cálculo de la clave privada $d$ es en lo que que se que se basa la seguridad de este criptosistema.